# Reversal Distance


ある順列に対して「反転(reversal)」とは、順列の連続した区間を選び、その区間内の要素の並びを逆順にする操作を指す。  

2つの順列πとσ(長さは同じ)に対し、πをσに変換するために必要な反転操作の最小回数を「反転距離(reversal distance)」$d_{rev}(π,σ)$と定義する。

与えられるのは、長さ10の順列からなる最大5組のペアである。
各ペアについて、最初の順列を2番目の順列に変換するための反転距離を求めよ。

出力は、各順列ペアに対応する反転距離を空白区切りで並べたものとする。



### 入力例

```text
1 2 3 4 5 6 7 8 9 10
3 1 5 2 7 4 9 6 10 8

3 10 8 2 5 4 7 1 6 9
5 2 3 1 7 4 10 8 6 9

8 6 7 9 4 1 3 10 2 5
8 2 7 6 9 1 5 3 10 4

3 9 10 4 1 8 6 7 5 2
2 9 8 5 1 7 3 4 6 10

1 2 3 4 5 6 7 8 9 10
1 2 3 4 5 6 7 8 9 10
```

### 出力例

```
9 4 5 7 0
```

# 解法

## (1) 問題をアルゴリズムの分野で考える

最小を求める → 動的計画法？ 最短路問題？

## (2) 適切な分野を想起する
順序は関係ない（遷移が定義されない）
→ **最短経路問題っぽい**

## (3) 分野から手法を想起する

最短経路アルゴリズム（幅優先探索, ダイクストラ、ベルマンフォールド…）

本問題は辺に重みがない → **幅優先探索**？


In [39]:
sample_input = """1 2 3 4 5 6 7 8 9 10
3 1 5 2 7 4 9 6 10 8

3 10 8 2 5 4 7 1 6 9
5 2 3 1 7 4 10 8 6 9

8 6 7 9 4 1 3 10 2 5
8 2 7 6 9 1 5 3 10 4

3 9 10 4 1 8 6 7 5 2
2 9 8 5 1 7 3 4 6 10

1 2 3 4 5 6 7 8 9 10
1 2 3 4 5 6 7 8 9 10"""

In [33]:
def parse_input(input_string: str) -> list[tuple[tuple[int], tuple[int]]]:
    lines = [line for line in input_string.splitlines() if line] # ignore empty lines

    # Pair up every two lines
    return [
        (
            tuple(map(int, line1.split())),
            tuple(map(int, line2.split())),
        )
        for line1, line2 in zip(lines[0::2], lines[1::2])
    ]


In [35]:
input_formatted = parse_input(sample_input)
print(input_formatted[:3])

[((1, 2, 3, 4, 5, 6, 7, 8, 9, 10), (3, 1, 5, 2, 7, 4, 9, 6, 10, 8)), ((3, 10, 8, 2, 5, 4, 7, 1, 6, 9), (5, 2, 3, 1, 7, 4, 10, 8, 6, 9)), ((8, 6, 7, 9, 4, 1, 3, 10, 2, 5), (8, 2, 7, 6, 9, 1, 5, 3, 10, 4))]


In [43]:
from collections import deque
from collections.abc import Generator

def generate_reversals(p: tuple[int, ...]) -> Generator:
    n = len(p)
    for i in range(n - 1):
        for j in range(i + 1, n):
            yield p[:i] + tuple(reversed(p[i:j + 1])) + p[j + 1:]


def reversal_distance_bfs(start: tuple[int, ...], goal: tuple[int, ...]) -> int:
    if start == goal:
        return 0

    # BFS queue: (現在の順列, 距離)
    queue = deque([(start, 0)])

    visited: set[tuple[int, ...]] = {start}

    while queue:
        seq, dist = queue.popleft()

        for q in generate_reversals(seq):
            if q in visited:
                continue

            if q == goal:
                return dist + 1

            visited.add(q)
            queue.append((q, dist + 1))

In [7]:
ans = [reversal_distance_bfs(a, b) for a, b in sample_input]
print(*ans)

9 4 5 7 0


## Test Dataset from Rosalind

In [ ]:
from pathlib import Path
input_text = Path("rosalind_rear.txt").read_text()
input_data = parse_input(input_text)
print(input_data[:2])

[((9, 7, 1, 3, 6, 2, 4, 5, 10, 8), (1, 9, 6, 7, 4, 3, 10, 2, 8, 5)), ((4, 6, 9, 5, 2, 1, 3, 10, 7, 8), (6, 10, 2, 1, 9, 8, 5, 7, 3, 4))]


In [ ]:
ans = [reversal_distance_bfs(a, b) for a, b in input_data]
print(*ans) # 2 min

9 6 6 3 6


## 双方向BFS

単純なBFS（単方向BFS）でも間に合いますが、二分もかかりました。  
単方向BFSではスタートからゴールに向けての一方向性でしたが、
スタートとゴールの両方の計算をして、途中で出会ったら終了する
**双方向性BFS**のほうが良さそうです。

単方向BFSでは、探索数が
$1 + b + b^2 + b^d$
となり、
分岐数bはだいたい$_{10}C_2 = 45$,
距離dは$7-10$程度なので、
探索数はおよそ$b^d = 45^10$となって相当に重たいです。


双方向BFSで計算すると、

$b^{d/2} + b^{d/2}$
となり、
探索の深さ（距離）が劇的に減るため、
高速に動くことが期待できます。

In [46]:
def reversal_distance_bidir_bfs(start: tuple[int], goal: tuple[int]) -> int:
    """
    Compute reversal distance using bidirectional BFS (unit edge costs).
    """
    if start == goal:
        return 0

    current_a: set[tuple[int]] = {start}
    current_b: set[tuple[int]] = {goal}

    visited_a: dict[tuple[int], int] = {start: 0}
    visited_b: dict[tuple[int], int] = {goal: 0}

    while current_a and current_b:
        # Expand the smaller frontier to reduce work
        if len(current_a) > len(current_b):
            current_a, current_b = current_b, current_a
            visited_a, visited_b = visited_b, visited_a

        next_a: set[tuple[int]] = set()
        for p in current_a:
            d = visited_a[p]
            for q in generate_reversals(p):
                if q in visited_a:
                    continue
                if q in visited_b:
                    return d + 1 + visited_b[q]
                visited_a[q] = d + 1
                next_a.add(q)

        current_a = next_a



In [ ]:
from pathlib import Path
input_text = Path("rosalind_rear.txt").read_text()
input_data = parse_input(input_text)
print(input_data[:2])
ans = [reversal_distance_bidir_bfs(a, b) for a, b in input_data]
print(*ans) # 1.3 sec !!

[((9, 7, 1, 3, 6, 2, 4, 5, 10, 8), (1, 9, 6, 7, 4, 3, 10, 2, 8, 5)), ((4, 6, 9, 5, 2, 1, 3, 10, 7, 8), (6, 10, 2, 1, 9, 8, 5, 7, 3, 4))]
9 6 6 3 6
